<a href="https://colab.research.google.com/github/ch00226855/CMP414765Spring2022/blob/main/Week12_SpamEmailDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Load the data from
# https://www.kaggle.com/datasets/venky73/spam-mails-dataset
from google.colab import files
uploaded = files.upload()

Saving spam_ham_dataset.csv to spam_ham_dataset (1).csv


In [20]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv('spam_ham_dataset.csv', index_col=[0])
df.head() # What is the first column?

,label,text,label_num
605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
4685,spam,"Subject: photoshop , windows , office . cheap ...",1
2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [23]:
df.sort_index() # Rows with small indices are ham emails. Rows with large indices are spam emails.

,label,text,label_num
0,ham,Subject: christmas tree farm pictures\r\n,0
1,ham,"Subject: vastar resources , inc .\r\ngary , pr...",0
2,ham,Subject: calpine daily gas nomination\r\n- cal...,0
3,ham,Subject: re : issue\r\nfyi - see note below - ...,0
4,ham,Subject: meter 7268 nov allocation\r\nfyi .\r\...,0
...,...,...,...
5166,spam,Subject: our pro - forma invoice attached\r\nd...,1
5167,spam,Subject: str _ rndlen ( 2 - 4 ) } { extra _ ti...,1
5168,spam,Subject: check me out !\r\n61 bb\r\nhey derm\r...,1
5169,spam,Subject: hot jobs\r\nglobal marketing specialt...,1


In [25]:
# Shape? Missing values? Value counts?
df.shape

(5171, 3)

In [35]:
# Display the entire content of an email
ind = 1234
print(df.loc[ind, 'text'])

Subject: final cp name change / merger list 7 / 00
this is the final list of counterparty name change and mergers for 7 / 00 .
the changes will be made in the global counterparty system as follows :
name changes - the current legal name and erms shortname will be overlayed on
monday , 7 / 24 , in the afternoon .
we want to update the system before bid week begins on tuesday . if anyone has
a problem with the erms shortnames being changed on monday , please notify us
immediately .
mergers / business changes - the old record will be inactivated on 8 / 1 / 00 .
marilyn colbert 3 6517
cheryl johnson 3 3103


In [31]:
df.isnull().sum()

label        0
text         0
label_num    0
dtype: int64

In [32]:
df['label'].value_counts()

ham     3672
spam    1499
Name: label, dtype: int64

In [33]:
# Train-test split
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.33, random_state=1357)

print('Training Data Shape:', df_train.shape)
print('Testing Data Shape: ', df_test.shape)

Training Data Shape: (3464, 3)
Testing Data Shape:  (1707, 3)


In [34]:
X_train = df_train['text'] 
y_train = df_train['label']
X_test = df_test['text'] 
y_test = df_test['label']

## How can we represent the emails as numbers?

- ASCII encoding: the meaning is lost
- Word embedding: use a vector to represent a word
- Using statistics: TF-IDF approach

### The TF-IDF Approach

- **Term Frequency (TF)**: How many times does each word appears?

- **Document Frequency (DF)**: How many documents contains this word?

- **TF-IDF metric**: TF / DF

In [36]:
# Example
ind = 1234
email = df.loc[ind, 'text']
print(email)

Subject: final cp name change / merger list 7 / 00
this is the final list of counterparty name change and mergers for 7 / 00 .
the changes will be made in the global counterparty system as follows :
name changes - the current legal name and erms shortname will be overlayed on
monday , 7 / 24 , in the afternoon .
we want to update the system before bid week begins on tuesday . if anyone has
a problem with the erms shortnames being changed on monday , please notify us
immediately .
mergers / business changes - the old record will be inactivated on 8 / 1 / 00 .
marilyn colbert 3 6517
cheryl johnson 3 3103


In [46]:
# for word in words:
#     if 'a' in word:
#         print(word)

final
name
change
final
counterparty
name
change
and
changes
made
global
counterparty
as
:
name
changes
legal
name
and
shortname
overlayed
on
monday
afternoon
want
update
tuesday
anyone
has
a
shortnames
changed
monday
please
us
immediately
changes
inactivated
.
marilyn


In [55]:
# Find out the frequency of word "final"
words = email.split(" ")
print(words)

words = [word.lower().strip() for word in words]
print(words)

word_freqs = {}

for word in words:
    if word not in word_freqs:
        word_freqs[word] = 1
    else:
        word_freqs[word] += 1
    
word_freqs['tuesday']

['Subject:', 'final', 'cp', 'name', 'change', '/', 'merger', 'list', '7', '/', '00\r\nthis', 'is', 'the', 'final', 'list', 'of', 'counterparty', 'name', 'change', 'and', 'mergers', 'for', '7', '/', '00', '.\r\nthe', 'changes', 'will', 'be', 'made', 'in', 'the', 'global', 'counterparty', 'system', 'as', 'follows', ':\r\nname', 'changes', '-', 'the', 'current', 'legal', 'name', 'and', 'erms', 'shortname', 'will', 'be', 'overlayed', 'on\r\nmonday', ',', '7', '/', '24', ',', 'in', 'the', 'afternoon', '.\r\nwe', 'want', 'to', 'update', 'the', 'system', 'before', 'bid', 'week', 'begins', 'on', 'tuesday', '.', 'if', 'anyone', 'has\r\na', 'problem', 'with', 'the', 'erms', 'shortnames', 'being', 'changed', 'on', 'monday', ',', 'please', 'notify', 'us\r\nimmediately', '.\r\nmergers', '/', 'business', 'changes', '-', 'the', 'old', 'record', 'will', 'be', 'inactivated', 'on', '8', '/', '1', '/', '00', '.\r\nmarilyn', 'colbert', '3', '6517\r\ncheryl', 'johnson', '3', '3103']
['subject:', 'final', '

1

## Model Construction

In [59]:
email1 = "how are you are you you"
email2 = "hello how you you are hello"
temp_data = [email1, email2]
temp = CountVectorizer()
vectors = temp.fit_transform(temp_data)
print(vectors)

# 0 - are
# 1 - hello
# 2 - how
# 3 - you

  (0, 2)	1
  (0, 0)	2
  (0, 3)	3
  (1, 2)	1
  (1, 0)	1
  (1, 3)	2
  (1, 1)	2


In [60]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3464, 40210)

In [61]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3464, 40210)

In [62]:
pd.DataFrame(X_train_tfidf)[0]

0         (0, 39920)\t0.02743184381260974\n  (0, 39909...
1         (0, 40046)\t0.06355447024579773\n  (0, 38726...
2         (0, 35782)\t0.22819066920050324\n  (0, 35632...
3         (0, 39909)\t0.04743370199590683\n  (0, 39368...
4         (0, 39926)\t0.060851176074157305\n  (0, 3992...
                              ...                        
3459      (0, 39909)\t0.060986604953790935\n  (0, 3916...
3460      (0, 39909)\t0.02755623396808801\n  (0, 39803...
3461      (0, 40187)\t0.0429528506709799\n  (0, 40182)...
3462      (0, 40047)\t0.11441975124347945\n  (0, 39909...
3463      (0, 38925)\t0.25833298980626385\n  (0, 38726...
Name: 0, Length: 3464, dtype: object

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train)
X_train_tfidf.shape

(3464, 40210)

In [64]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [65]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [66]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1191   17]
 [   6  493]]


In [68]:
print(metrics.accuracy_score(y_test, predictions))

0.9865260691271236


In [67]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1208
        spam       0.97      0.99      0.98       499

    accuracy                           0.99      1707
   macro avg       0.98      0.99      0.98      1707
weighted avg       0.99      0.99      0.99      1707



In [69]:
text_clf.predict(['Congratulations , you won @ new stage '])[0]

'spam'

In [70]:
text_clf.predict(['Subject: Final Exam Date; \n Hi Everyone, \n The final exam of CMP 464/788 is scheduled for Wednesday, May 18th. \n Best, \n Liang Zhao'])[0]

'ham'

## How about logistic regression?

In [73]:
from sklearn.linear_model import LogisticRegression
tfidf_vectorizer2 = TfidfVectorizer()
logistic_model = LogisticRegression()
classifier2 = Pipeline([('tfidf', tfidf_vectorizer2), 
                        ('lr', logistic_model)])


In [74]:
classifier2.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lr', LogisticRegression())])

In [75]:
test_predictions = classifier2.predict(X_test)

In [78]:
metrics.accuracy_score(y_test, test_predictions)

0.9824253075571178

In [79]:
metrics.confusion_matrix(y_test, test_predictions)

array([[1187,   21],
       [   9,  490]])